![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

# Full Script

In [39]:
import pandas as pd
import numpy as np
import datetime as dt

# Start coding here...
bank_df = pd.read_csv('bank_marketing.csv')

#########################################################

# Start coding here...
bank_df = pd.read_csv('bank_marketing.csv')

#########################################################
client = bank_df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default','housing', 'loan']]
client = client.rename(columns={'client_id': 'id'})
            
client['job'] = client['job'].str.replace('.', '')
client['education'] = client['education'].str.replace('.', '_')
client['education'].replace(to_replace='unknown', value=np.NaN, inplace=True)

# Replace unknown values with nan
client.loc[client['credit_default'] == 'unknown', 'credit_default'] = np.nan
# Define the mapping for conversion
mapping = {'no': False, 'yes': True}
# Replace the values in the DataFrame column and convert to boolean
client['credit_default'] = client['credit_default'].replace(mapping).astype('boolean')


# Replace unknown values with nan
client.loc[client['housing'] == 'unknown', 'housing'] = np.nan
# Replace the values in the DataFrame column and convert to boolean
client['housing'] = client['housing'].replace(mapping).astype('boolean')


# Replace unknown values with nan
client.loc[client['loan'] == 'unknown', 'loan'] = np.nan
# Replace the values in the DataFrame column and convert to boolean
client['loan'] = client['loan'].replace(mapping).astype('boolean')

#########################################################
campaign = bank_df[['client_id', 'campaign', 'duration', 'pdays','previous', 'poutcome', 'y']]


campaign_new_columns = {
    'duration':'contact_duration',
    'previous':'previous_campaign_contacts',
    'y':'campaign_outcome',
    'poutcome':'previous_outcome',
    'campaign':'number_contacts'
}

campaign = campaign.rename(columns=campaign_new_columns)


# campaign['previous_outcome'].replace(to_replace=['success','failure', 'nonexistent'], value=[1,0, np.NaN], inplace=True)
# campaign['previous_outcome'].replace({'success':1,'failure':0, 'nonexistent':np.nan}, inplace=True)

campaign['campaign_id'] = 1

# Replace unknown values with nan
campaign.loc[campaign['previous_outcome'] == 'nonexistent', 'previous_outcome'] = np.nan
# Define the mapping for conversion
mapping = {'success':1,'failure':0}
# Replace the values in the DataFrame column and convert to boolean
campaign['previous_outcome'] = campaign['previous_outcome'].replace(mapping).astype('boolean')

# Define the mapping for conversion
mapping = {'yes':1,'no':0}
# Replace the values in the DataFrame column and convert to boolean
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace(mapping).astype('boolean')

current_year = dt.date.today().year
campaign['last_contact_date'] = pd.to_datetime(
    bank_df['day'].astype(str) + ' ' + bank_df['month'] + ' ' + str(current_year),
    format='%d %b %Y'
)

#########################################################
economics = bank_df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

economics_new_columns = {
    'euribor3m':'euribor_three_months',
    'nr_employed':'number_employed'
}
economics = economics.rename(columns=economics_new_columns)

#########################################################
client.to_csv('client.csv', index=False,sep=',')
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)
#########################################################

client_table = """
    CREATE TABLE client (
        id SERIAL PRIMARY KEY,
        age INT,
        job TEXT,
        marital TEXT,
        education TEXT,
        credit_default BOOLEAN,
        housing BOOLEAN,
        loan BOOLEAN
    )
    \COPY client FROM 'client.csv' DELIMITER ',' CSV HEADER;  
"""

campaign_table = """
    CREATE TABLE campaign (
        client_id serial REFERENCES client (id),
        campaign_id serial PRIMARY KEY,
        number_contacts INT,
        contact_duration INT,
        pdays INT,
        previous_campaign_contacts INT,
        previous_outcome BOOLEAN,
        campaign_outcome BOOLEAN,
        last_contact_date DATE,
    );
    \COPY campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;  
"""
economics_table = """
    CREATE TABLE economics (
        client_id SERIAL PRIMARY KEY,
        emp_var_rate FLOAT,
        cons_price_idx FLOAT,
        euribor_three_months FLOAT,
        number_employed FLOAT,
        client_id serial REFERENCES client (id)
    );
    \COPY economics FROM 'economics.csv' DELIMITER ',' CSV HEADER;  
"""

In [4]:

tuple((1, 2))

(1, 2)